<a href="https://colab.research.google.com/github/KonradGonrad/PyTorch-deep-learning/blob/main/Kopia_notatnika_05_pytorch_going_modular_exercise_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 05. PyTorch Going Modular Exercises

Welcome to the 05. PyTorch Going Modular exercise template notebook.

There are several questions in this notebook and it's your goal to answer them by writing Python and PyTorch code.

> **Note:** There may be more than one solution to each of the exercises, don't worry too much about the *exact* right answer. Try to write some code that works first and then improve it if you can.

## Resources and solutions

* These exercises/solutions are based on [section 05. PyTorch Going Modular](https://www.learnpytorch.io/05_pytorch_going_modular/) of the Learn PyTorch for Deep Learning course by Zero to Mastery.

**Solutions:**

Try to complete the code below *before* looking at these.

* See a live [walkthrough of the solutions (errors and all) on YouTube](https://youtu.be/ijgFhMK3pp4).
* See an example [solutions notebook for these exercises on GitHub](https://github.com/mrdbourke/pytorch-deep-learning/blob/main/extras/solutions/05_pytorch_going_modular_exercise_solutions.ipynb).

## 1. Turn the code to get the data (from section 1. Get Data) into a Python script, such as `get_data.py`.

* When you run the script using `python get_data.py` it should check if the data already exists and skip downloading if it does.
* If the data download is successful, you should be able to access the `pizza_steak_sushi` images from the `data` directory.

In [1]:
%%writefile get_data.py
# YOUR CODE HERE
from pathlib import Path
import requests
import zipfile
import os

DATA_PATH = Path('data/')
IMAGES_PATH = DATA_PATH / 'pizza_steak_sushi'

if IMAGES_PATH.is_dir():
  print(f'{IMAGES_PATH} already exists.')
else:
  IMAGES_PATH.mkdir(parents=True, exist_ok=True)

  with open(DATA_PATH / 'pizza_steak_sushi.zip', 'wb') as f:
    request = requests.get('https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip')
    f.write(request.content)

  with zipfile.ZipFile(DATA_PATH / 'pizza_steak_sushi.zip', 'r') as zipf:
    zipf.extractall(IMAGES_PATH)

  os.remove(DATA_PATH / 'pizza_steak_sushi.zip')

Writing get_data.py


In [2]:
# Example running of get_data.py
!python get_data.py

## 2. Use [Python's `argparse` module](https://docs.python.org/3/library/argparse.html) to be able to send the `train.py` custom hyperparameter values for training procedures.
* Add an argument flag for using a different:
  * Training/testing directory
  * Learning rate
  * Batch size
  * Number of epochs to train for
  * Number of hidden units in the TinyVGG model
    * Keep the default values for each of the above arguments as what they already are (as in notebook 05).
* For example, you should be able to run something similar to the following line to train a TinyVGG model with a learning rate of 0.003 and a batch size of 64 for 20 epochs: `python train.py --learning_rate 0.003 batch_size 64 num_epochs 20`.
* **Note:** Since `train.py` leverages the other scripts we created in section 05, such as, `model_builder.py`, `utils.py` and `engine.py`, you'll have to make sure they're available to use too. You can find these in the [`going_modular` folder on the course GitHub](https://github.com/mrdbourke/pytorch-deep-learning/tree/main/going_modular/going_modular).

In [3]:
%%writefile model_builder.py

import torch
from torch import nn

class TinyVGG(nn.Module):
  def __init__(self,
               input_channels: int,
               hidden_channels: int,
               output_channels: int):
    super().__init__()
    self.layer_1 = nn.Sequential(
        nn.Conv2d(in_channels=input_channels,
                  out_channels=hidden_channels,
                  kernel_size=3,
                  stride=1,
                  padding=1),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_channels,
                  out_channels=hidden_channels,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2,
                     padding=0)
    )
    self.layer_2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_channels,
                  out_channels=hidden_channels,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_channels,
                  out_channels=hidden_channels,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2,
                     stride=2,
                     padding=0)
    )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features = hidden_channels * 13 * 13,
                  out_features=output_channels)
    )

  def forward(self, x):
    x = self.layer_1(x)
    #print(x.shape)
    x = self.layer_2(x)
    #print(x.shape)
    x = self.classifier(x)
    return x

Writing model_builder.py


In [4]:
%%writefile data_setup.py

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

from get_data import IMAGES_PATH

TRAIN_DIR = IMAGES_PATH / 'train'
TEST_DIR = IMAGES_PATH / 'test'
BATCH_SIZE = 1

simple_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
])

def prepare_data(transforms: transforms = simple_transform,
                 train_dir: str = TRAIN_DIR,
                 test_dir: str = TEST_DIR,
                 batch_size: int = BATCH_SIZE
                 ):
  """
    Function that prepares data and transform it into dict of train_data, test_data, train_dataloader, test_dataloader and class_names.
    Returns dict
  """
  prepared_data = {
      'class_names': None,
      'train_data' : None,
      'test_data' : None,
      'train_dataloader' : None,
      'test_dataloader' : None
  }

  train_data = datasets.ImageFolder(root=train_dir,
                                    transform = transforms,
                                    target_transform= None)

  test_data = datasets.ImageFolder(root=test_dir,
                                   transform=transforms,
                                   target_transform=None)

  class_names = train_data.classes

  train_dataloader = DataLoader(train_data,
                                batch_size=batch_size,
                                shuffle=True)

  test_dataloader = DataLoader(test_data,
                               batch_size=batch_size,
                               shuffle=False)

  prepared_data['class_names'] = class_names
  prepared_data['train_data'] = train_data
  prepared_data['test_data'] = test_data
  prepared_data['train_dataloader'] = train_dataloader
  prepared_data['test_dataloader'] = test_dataloader

  return prepared_data

Writing data_setup.py


In [5]:
%%writefile engine.py

# YOUR CODE HERE
from torch import nn
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

def train_step(model: nn.Module,
               train_dataloader: DataLoader,
               loss_fn: nn.Module,
               optimizer: torch.optim,
               device: torch.device):
  model.train()

  train_loss, train_acc = 0, 0

  for batch, (X, y) in enumerate(train_dataloader):
    X, y = X.to(device), y.to(device)
    # Forward pass
    y_logit = model(X)
    # Calculate the loss
    loss = loss_fn(y_logit, y)
    train_loss += loss.item()
    # Optimizer zero grad
    optimizer.zero_grad()
    # loss backward
    loss.backward()
    # Optimizer step
    optimizer.step()
    # Calculate accuracy
    y_label = torch.argmax(torch.softmax(y_logit, dim = 1), dim = 1)
    train_acc += ((y_label == y).sum().item() / len(y_label))
  train_loss /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  return train_loss, train_acc

def test_step(model: nn.Module,
              test_dataloader: DataLoader,
              loss_fn: nn.Module,
              device: torch.device):
  test_loss, test_acc = 0, 0

  model.eval()

  with torch.inference_mode():
    for batch, (X, y) in enumerate(test_dataloader):
      X, y = X.to(device), y.to(device)
      # Forward pass
      y_logit = model(X)
      # Calculate the loss
      loss = loss_fn(y_logit, y)
      test_loss += loss.item()
      # Calculate the acc
      y_label = torch.argmax(torch.softmax(y_logit, dim=1), dim=1)
      test_acc += ((y_label == y).sum().item() / len(y_label))
  test_loss /= len(test_dataloader)
  test_acc /= len(test_dataloader)

  return test_loss, test_acc

def train(model: nn.Module,
          test_dataloader: DataLoader,
          train_dataloader: DataLoader,
          optimizer: torch.optim,
          loss_fn: nn.Module,
          epochs: int,
          device: torch.device):

  results = {
      'epoch' : [],
      'train_loss': [],
      'train_acc' : [],
      'test_loss' : [],
      'test_acc' : []
  }

  for epoch in tqdm(range(epochs)):
    train_loss, train_acc = train_step(model=model,
                                       train_dataloader=train_dataloader,
                                       loss_fn=loss_fn,
                                       optimizer=optimizer,
                                       device=device)

    test_loss, test_acc = test_step(model=model,
                                    test_dataloader=test_dataloader,
                                    loss_fn=loss_fn,
                                    device=device)

    print(
        f'Epoch: {epoch + 1} |'
        f'train_loss: {train_loss:.4f} |'
        f'train_accuracy: {train_acc:.4f} |'
        f'test_loss: {test_loss:.4f} |'
        f'test_accuracy: {test_acc:.4f}'
    )

    results['epoch'].append(epoch + 1)
    results['train_loss'].append(train_loss)
    results['train_acc'].append(train_acc)
    results['test_loss'].append(test_loss)
    results['test_acc'].append(test_acc)

  return results

Writing engine.py


In [9]:
%%writefile utils.py

import torch
from pathlib import Path
from torch import nn

def save_model(model: nn.Module,
               target_dir: str,
               model_name: str):

  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  assert model_name.endswith('.pth') or model_name.endswith('.pt'), 'model_name needs to end with ".pth" or ".ph"'
  model_save_path = target_dir_path / model_name

  torch.save(obj=model.state_dict(),
             f=model_save_path)

Overwriting utils.py


In [11]:
%%writefile train.py

import os
import torch
import engine, data_setup, model_builder, utils
from torchvision import transforms
import argparse
from pathlib import Path
from torch import nn

torch.manual_seed(42)
torch.cuda.manual_seed(42)

def parse_args():
  parser = argparse.ArgumentParser(description='Parser to change default parameters')

  parser.add_argument('--num_epochs', type=int, default=5, help='How much epochs')
  parser.add_argument('--batch_size', type=int, default=1,
                      help='How much samples per batch')
  parser.add_argument('--hidden_units', type=int, default=10,
                      help='Number of hidden layers in model')
  parser.add_argument('--learning_rate', type=float, default=0.001,
                      help='Float number of learning rate in optimizer. Deafult: 0.001')
  parser.add_argument('--train_dir', type=str, default=Path('data/pizza_steak_sushi/train'),
                      help='Path of the train direction')
  parser.add_argument('--test_dir', type=str, default=Path('data/pizza_steak_sushi/test'),
                      help='Path of the test direction')
  parser.add_argument('--device', type = str, choices=['cuda', 'cpu'], default= 'cuda' if torch.cuda.is_available() else 'cpu', )
  parser.add_argument('--model_name', type=str, default='model_0.pth')

  return parser.parse_args()

def main():
  args = parse_args()

  MODEL_NAME = args.model_name
  NUM_EPOCHS = args.num_epochs
  BATCH_SIZE = args.batch_size
  HIDDEN_UNITS = args.hidden_units
  LEARNING_RATE = args.hidden_units

  TRAIN_DIR = args.train_dir
  TEST_DIR = args.test_dir

  DEVICE  = args.device
  SIMPLE_TRANSFORM = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.ToTensor()
  ])

  prepared_data = data_setup.prepare_data(transforms=SIMPLE_TRANSFORM,
                                          train_dir=TRAIN_DIR,
                                          test_dir=TEST_DIR,
                                          batch_size=BATCH_SIZE)

  train_dataloader = prepared_data['train_dataloader']
  test_dataloader = prepared_data['test_dataloader']
  class_names = prepared_data['class_names']

  model_0 = model_builder.TinyVGG(input_channels=3,
                                hidden_channels=HIDDEN_UNITS,
                                output_channels=len(class_names))

  loss_fn = nn.CrossEntropyLoss()
  optimizer = torch.optim.Adam(params=model_0.parameters(),
                              lr=LEARNING_RATE)

  engine.train(model=model_0,
              test_dataloader = test_dataloader,
              train_dataloader=train_dataloader,
              optimizer=optimizer,
              loss_fn=loss_fn,
              epochs=NUM_EPOCHS,
              device=DEVICE)

  SAVE_PATH = Path('models/')

  utils.save_model(model=model_0,
                   target_dir=SAVE_PATH,
                   model_name = MODEL_NAME
                   )


if __name__ == '__main__':
    main()

Overwriting train.py


In [13]:
# Example running of train.py
!python train.py --num_epochs 5 --batch_size 128 --hidden_units 128 --learning_rate 0.0003

data/pizza_steak_sushi already exists.
  0% 0/5 [00:00<?, ?it/s]Epoch: 1 |train_loss: 5664906834483506839552.0000 |train_accuracy: 0.3137 |test_loss: 2305919570407359250432.0000 |test_accuracy: 0.3333
 20% 1/5 [00:34<02:16, 34.08s/it]Epoch: 2 |train_loss: 941373989679116845056.0000 |train_accuracy: 0.3485 |test_loss: 29146891464375386767360.0000 |test_accuracy: 0.2533
 40% 2/5 [01:07<01:41, 33.87s/it]Epoch: 3 |train_loss: 13535035847931112456192.0000 |train_accuracy: 0.3187 |test_loss: 37238945904773300224.0000 |test_accuracy: 0.4133
 60% 3/5 [01:42<01:08, 34.14s/it]Epoch: 4 |train_loss: 33418226561135411200.0000 |train_accuracy: 0.3276 |test_loss: 2756654871229759488.0000 |test_accuracy: 0.3867
 80% 4/5 [02:15<00:33, 33.75s/it]Epoch: 5 |train_loss: 2417195224682987520.0000 |train_accuracy: 0.2050 |test_loss: 1509148703477202944.0000 |test_accuracy: 0.2933
100% 5/5 [02:56<00:00, 35.36s/it]


## 3. Create a Python script to predict (such as `predict.py`) on a target image given a file path with a saved model.

* For example, you should be able to run the command `python predict.py some_image.jpeg` and have a trained PyTorch model predict on the image and return its prediction.
* To see example prediction code, check out the [predicting on a custom image section in notebook 04](https://www.learnpytorch.io/04_pytorch_custom_datasets/#113-putting-custom-image-prediction-together-building-a-function).
* You may also have to write code to load in a trained model.

In [42]:
%%writefile predict.py
# YOUR CODE HERE
import torchvision
import argparse
import matplotlib.pyplot as plt
import torch
import model_builder
from torchvision import transforms
from PIL import Image
from pathlib import Path

def parse_args():
  parser = argparse.ArgumentParser()

  parser.add_argument('--image_path', type = str, default=None)
  parser.add_argument('--class_names', type=list, default=['pizza', 'steak', 'sushi'])
  parser.add_argument('--model_path', type=str, default=Path('models/model_0.pth'))
  return parser.parse_args()

def main():
  arg = parse_args()

  IMAGE_PATH = arg.image_path
  CLASS_NAMES = arg.class_names
  MODEL_PATH = arg.model_path

  SIMPLE_TRANSFORM = torchvision.transforms.Compose([
      transforms.Resize(size=(64, 64)),
      transforms.ToTensor()
  ])

  target_image = Image.open(str(IMAGE_PATH))
  target_image = SIMPLE_TRANSFORM(target_image)
  target_image /= 255

  model = model_builder.TinyVGG(input_channels=3,
                        hidden_channels=128,
                        output_channels=3)
  model.load_state_dict(torch.load(MODEL_PATH, weights_only=True))

  model.eval()
  with torch.inference_mode():
    target_image = target_image.unsqueeze(dim=0)
    y_logit = model(target_image)

  y_pred_label = torch.argmax(torch.softmax(y_logit, dim=1), dim=1)

  """  plt.imshow(target_image.squeeze().permute(1, 2, 0))
    if CLASS_NAMES != None:
      title = f'Prediction: {CLASS_NAMES[y_pred_label]}'
    else:
      title = f'Prediction: {y_pred_label}'
    plt.title(title)
    plt.axis("off")
    plt.show()
  """
  if CLASS_NAMES != None:
    prediction = f"Prediction: {CLASS_NAMES[y_pred_label]}"
  else:
    prediction = f"Prediction: {y_pred_label}"

  print(prediction)

if __name__ == '__main__':
  main()

Overwriting predict.py


In [43]:
# Example running of predict.py
!python predict.py --image data/pizza_steak_sushi/test/sushi/175783.jpg

Prediction: pizza
